# DAY 2

# Downloading the fastq files

Make a new directory on your computer for todays project.

On my laptop I am using:

    "/Users/alexis/Desktop/bd2k/"
    
Inside of the BD2K folder I will make a folder to save the fastq files.

    "/Users/alexis/Desktop/bd2k/fastq"
  
`curl -O https://users.soe.ucsc.edu/~bsaintjo/sacCer3_aligned/SRR5494631.fastq.gz`

`curl -O https://users.soe.ucsc.edu/~bsaintjo/sacCer3_aligned/SRR5494630.fastq.gz`

`curl -O https://users.soe.ucsc.edu/~bsaintjo/sacCer3_aligned/SRR5494629.fastq.gz`


`curl -O https://users.soe.ucsc.edu/~bsaintjo/sacCer3_aligned/SRR5494628.fastq.gz`


Each file represents a condition that we want to test on.

euploid:

    SRR5494631    SRR5494630
    
aneuploid:

    SRR5494629    SRR5494628 

# Running kallisto

https://pachterlab.github.io/kallisto/download

Download the latest version of `kallisto` for your Operating System and place it into the directory

### Index files
kallisto normally requires an index relating the genes to the sequence however, the index for our genome has already been built, so download the index and place it into the same directory.

Download the index
https://github.com/pachterlab/kallisto-transcriptome-indices/releases/download/94/Saccharomyces_cerevisiae.R64-1-1.cdna.all.release-94_k31.idx.gz

#### Running kallisto
There are a total of 4 samples and you will need to run kallisto on each sample individually.

    Usage: kallisto quant [arguments] FASTQ-files

    arguments:
    -i, --index=STRING            Filename for the kallisto index to be used for
                              quantification
                              
    -o, --output-dir=STRING       Directory to write output to


    -b, --bootstrap-samples=INT   Number of bootstrap samples (default: 0)


Make sure you are in your fastq directory.
Also, you need to make a `bd2k/kallisto_outputs directory` to put the kallisto outputs in

##### euploid samples

`kallisto quant -i /Users/alexis/Desktop/bd2k/Saccharomyces_cerevisiae.R64-1-1.cdna.all.release-94_k31.idx -o /Users/alexis/Desktop/bd2k/kallisto_outputs/SRR5494631 --single -b 100 -l 50 -s 0 -t 2  SRR5494631.fastq.gz `

`kallisto quant -i /Users/alexis/Desktop/bd2k/Saccharomyces_cerevisiae.R64-1-1.cdna.all.release-94_k31.idx -o /Users/alexis/Desktop/bd2k/kallisto_outputs/SRR5494630 --single -b 100 -l 50 -s 0 -t 2  SRR5494630.fastq.gz`
##### aneuploid samples

`kallisto quant -i /Users/alexis/Desktop/bd2k/Saccharomyces_cerevisiae.R64-1-1.cdna.all.release-94_k31.idx -o /Users/alexis/Desktop/bd2k/kallisto_outputs/SRR5494629 --single -b 100 -l 50 -s 0 -t 2  SRR5494629.fastq.gz`

`kallisto quant -i /Users/alexis/Desktop/bd2k/Saccharomyces_cerevisiae.R64-1-1.cdna.all.release-94_k31.idx -o /Users/alexis/Desktop/bd2k/kallisto_outputs/SRR5494628 --single -b 100 -l 50 -s 0 -t 2  SRR5494628.fastq.gz`

# Using pandas to compare the two sets

You should now have four folders, with each containing an `abundance.tsv` file. Using pandas we can now load the data using commands similar to what you have done before. Look at the previous notebooks and start to use `pandas` to process the data and find differentially expressed genes.

Lets look at a single output file...

In [6]:
import pandas as pd
SRR5494628 = pd.read_csv("/Users/alexis/Desktop/bd2k/kallisto_outputs/SRR5494628/abundance.tsv", sep="\t")
SRR5494628.head()

,target_id,length,eff_length,est_counts,tpm
0,YBR024W_mRNA,906,707.00000,544.0000,36.9940
1,YDL245C_mRNA,1704,1505.00000,902.1420,28.8197
2,YBR232C_mRNA,360,161.00000,46.2893,13.8231
3,YDR320W-B_mRNA,138,6.07083,0.0000,0.0000
4,YBR021W_mRNA,1902,1703.00000,426.0000,12.0267


We are mainly interested in the target_id and tpm column. In order to make processing easier with `pandas` we can use the following shell commands to combine the columns we want from the four files into one main file. Take a look at these commands and see if you recognize what is happening. Try using `man` to look at what the command actually does.

```
cd kallisto_outputs/

paste */abundance.tsv | cut -f 1,2,5,10,15,20 > tpms_all_samples.tsv

ls -1 */abundance.tsv | perl -ne 'chomp $_; if ($_ =~ /(\S+)\/abundance\.tsv/){print "\t$1"}' | perl -ne 'print "target_id\tlength$_\n"' > header.tsv

cat header.tsv tpms_all_samples.tsv | grep -v "tpm" > tpms_all_samples.tsv2

mv tpms_all_samples.tsv2 tpms_all_samples.tsv

rm -f header.tsv
```

Now the 4 files are combined into onw and the sample names are the column names for their respective columns.

In [7]:
tpms_all_samples= pd.read_csv("/Users/alexis/Desktop/bd2k/kallisto_outputs/tpms_all_samples.tsv", sep="\t")
tpms_all_samples.head()

,target_id,length,SRR5494628,SRR5494629,SRR5494630,SRR5494631
0,YBR024W_mRNA,906,36.9940,30.9409,37.3838,38.7345
1,YDL245C_mRNA,1704,28.8197,30.2001,12.2243,12.9081
2,YBR232C_mRNA,360,13.8231,10.0286,13.3590,15.9174
3,YDR320W-B_mRNA,138,0.0000,0.0000,0.0000,0.0000
4,YBR021W_mRNA,1902,12.0267,10.0594,17.3070,17.2506


In [8]:
#lets make lists to map the sample names to the condition
euploid = ['SRR5494631','SRR5494630']
aneuploid = ['SRR5494629','SRR5494628']

In [13]:
tpms_all_samples['euploid_average'] = tpms_all_samples[euploid].astype(float).mean(axis=1)
tpms_all_samples.head()

,target_id,length,SRR5494628,SRR5494629,SRR5494630,SRR5494631,euploid_average
0,YBR024W_mRNA,906,36.9940,30.9409,37.3838,38.7345,38.05915
1,YDL245C_mRNA,1704,28.8197,30.2001,12.2243,12.9081,12.56620
2,YBR232C_mRNA,360,13.8231,10.0286,13.3590,15.9174,14.63820
3,YDR320W-B_mRNA,138,0.0000,0.0000,0.0000,0.0000,0.00000
4,YBR021W_mRNA,1902,12.0267,10.0594,17.3070,17.2506,17.27880


Now that you know how to take the average of the 2 euploid samples, you can find the average of the two aneuploid samples. Then you can compare the average transcript expression between each condition. Try taking the difference or the ratio of the two.  